In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=""

env: CUDA_VISIBLE_DEVICES=""


In [2]:
from torchsummary import summary

In [23]:
from vit import PatchEmbeddings,  ViTForClassfication
from data import prepare_data
from torchsummary import summary

In [4]:
trainloader, testloader, classes = prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [5]:
for X, y in trainloader:
    break
print(X.shape, y.shape)

torch.Size([4, 3, 32, 32]) torch.Size([4])


In [6]:
config = {
    "patch_size": 4,  # Input image size: 32x32 -> 8x8 patches
    "hidden_size": 48,
    "num_hidden_layers": 4,
    "num_attention_heads": 4,
    "intermediate_size": 4 * 48, # 4 * hidden_size
    "hidden_dropout_prob": 0.0,
    "attention_probs_dropout_prob": 0.0,
    "initializer_range": 0.02,
    "image_size": 32,
    "num_classes": 10, # num_classes of CIFAR10
    "num_channels": 3,
    "qkv_bias": True,
    "use_faster_attention": True,
}

In [7]:
from torch import nn
from einops.layers.torch import Rearrange
from torch import Tensor
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels = 3, patch_size = 8, emb_size = 128):
        self.patch_size = patch_size
        super().__init__()
        self.projection = nn.Sequential(
            # break-down the image in s1 x s2 patches and flat them
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_size, p2=patch_size),
            nn.Linear(patch_size * patch_size * in_channels, emb_size)
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.projection(x)
        return x

In [11]:
pe = PatchEmbedding(in_channels = 3, patch_size = 4, emb_size = 48)

summary(pe, input_size=(3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Rearrange-1               [-1, 64, 48]               0
            Linear-2               [-1, 64, 48]           2,352
Total params: 2,352
Trainable params: 2,352
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB): 0.01
Estimated Total Size (MB): 0.07
----------------------------------------------------------------


In [13]:
pes = PatchEmbeddings(config)
summary(pes, input_size=(3, 32, 32))

torch.Size([2, 48, 8, 8])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 48, 8, 8]           2,352
Total params: 2,352
Trainable params: 2,352
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 0.01
Estimated Total Size (MB): 0.04
----------------------------------------------------------------


In [9]:
pe(X).shape

torch.Size([4, 48, 8, 8])


torch.Size([4, 64, 48])

In [31]:
model = ViTForClassfication(config)
model(X)[0].shape

torch.Size([4, 48, 8, 8])


torch.Size([4, 10])

In [29]:
summary(model, (3, 32, 32))

torch.Size([2, 48, 8, 8])


AttributeError: 'list' object has no attribute 'size'

In [21]:
model(X)[0]

torch.Size([4, 48, 8, 8])


tensor([[ 0.0184,  0.0023, -0.0005,  0.0129,  0.0008, -0.0047, -0.0076,  0.0034,
         -0.0115,  0.0039],
        [ 0.0040,  0.0027,  0.0024,  0.0140, -0.0015, -0.0074, -0.0022,  0.0015,
         -0.0108, -0.0017],
        [ 0.0079,  0.0035,  0.0016,  0.0147, -0.0011, -0.0062, -0.0032,  0.0002,
         -0.0110, -0.0008],
        [ 0.0138,  0.0039,  0.0031,  0.0160, -0.0008, -0.0034, -0.0035,  0.0012,
         -0.0086,  0.0015]], grad_fn=<AddmmBackward0>)